## **Generative Adversarial Networks (GANs)**

Linear Regression is a simple supervised learning algorithm used for predicting a continuous target variable based on one or more features. It assumes a linear relationship between the independent variables and the dependent variable.



**Imports**

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


**Data Loading**

In [ ]:
# Example: Load MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype('float32') - 127.5) / 127.5  # Normalize to [-1, 1]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)


**Model Building**

In [ ]:
# Generator Model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=100),
        layers.Reshape((4, 4, 8)),
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(1, kernel_size=4, strides=7, padding='same', activation='tanh')
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=(28, 28, 1)),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy')


**Training the GAN**

In [ ]:
# GAN Model (Generator + Discriminator)
discriminator.trainable = False
gan_input = layers.Input(shape=(100,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)
gan = tf.keras.models.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Training loop
batch_size = 64
epochs = 10000
for epoch in range(epochs):
    # Train discriminator
    real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
    fake_images = generator.predict(np.random.randn(batch_size, 100))
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    discriminator.train_on_batch(real_images, real_labels)
    discriminator.train_on_batch(fake_images, fake_labels)

    # Train generator via GAN
    noise = np.random.randn(batch_size, 100)
    misleading_labels = np.ones((batch_size, 1))
    gan.train_on_batch(noise, misleading_labels)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: GAN training step completed.")


**Visualizations**

In [ ]:
# Generate and visualize new images
noise = np.random.randn(16, 100)
generated_images = generator.predict(noise)

plt.figure(figsize=(10, 10))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')
plt.suptitle("Generated Images")
plt.show()
